# Vehicle Recall Data - David's Work with CMPL and Mileage
### Initial Questions
* What Make/Model is being recalled?
    * See Fig 1, Fig 2, and Fig 3
    * Honda, Ford, Blue Bird most common
* What specific problems led to the recall?
    * Most common by far is airbags (Fig 5)
* Is this correlated to an increase in defects or better QA?
* At what mileage did we start seeing the problem?
    * Average Miles at Failure: 68334.08213141191
* Did an increase in complaints lead to a recall?
    * Generally, as the number of recalls go up, the number of complaints also go up (Fig 4)

# Required Code

This is the code required to load the datasets we need

In [10]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import re

data_path = 'data'
fields_path = 'fields'

# Read and parse a file containing column names for a dataset.
# Returns a list of strings (the column names).
def load_column_names(columns_file_name):
    file = open(columns_file_name)
    column_names = []
    readingFields = False
    for line in file:
        if readingFields:
            if re.match('^[0-9]', line):
                split_line = line.split()
                column_names.append(split_line[1])
        elif line.startswith('Field#'):
            readingFields = True
    file.close()
    return column_names
#}

# Load a dataest by its abbreviated names (ex: RCL).
def load_data_file(name, encoding):
    
    # Read the column names from a separate file.
    fields_file_name = fields_path + '/' + name + '.txt';
    names = load_column_names(fields_file_name)
    
    # Read the data from a CSV file.
    data_file_name = 'FLAT_' + name + '.txt';
    df = pd.read_csv(data_path + '/' + data_file_name,
                     delimiter='\t', header=None, encoding=encoding, names=names)
    
    return df
#}


In [3]:
# Load complaints dataset.
cmpl_df = load_data_file('CMPL', encoding='iso-8859-1');

# Load recalls dataset.
rcl_df = load_data_file('RCL', encoding='latin1');

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (15,22,26,32,33,34,35,36,38,39,40,41,42,43,44,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


# Data Exploration

In [ ]:

#plt.figure()
#cmpl_df['MILES'].plot(kind='box')
#plt.gcf()
#cmpl_df['MILES'].describe()

col_miles = cmpl_df['MILES'].dropna();


#cmpl_df[(cmpl_df['MILES'] >= 99) & (cmpl_df['MILES'] <= 101)]["MILES"]
#cmpl_df[(cmpl_df['MILES'] < 0)]["MILES"]

clean_miles = cmpl_df[(cmpl_df["MILES"] >= 0) & (cmpl_df["MILES"] < 300000)]["MILES"]

clean_miles = clean_miles / 1000;

clean_df = cmpl_df[(cmpl_df["MILES"] >= 0) & (cmpl_df["MILES"] < 300000)]

#clean_miles = cmpl_df['VEH_SPEED'].dropna();
#clean_miles = pd.to_numeric(cmpl_df['NUM_CYLS'], errors='coerce').dropna()
#clean_miles = clean_miles[(clean_miles >= 2) & (clean_miles <= 10)]
#clean_miles = cmpl_df[(cmpl_df['VEH_SPEED'] >= 140) & (cmpl_df['VEH_SPEED'] < 1000)]['VEH_SPEED']

clean_miles = cmpl_df['DEATHS']
clean_miles = clean_miles[(clean_miles > 10) & (clean_miles <= 99)]


plt.figure()
clean_miles.hist(bins=20, color='#FF5500')
#clean_df.plot(kind='scatter', x='MILES', y='OCCURENCES');
plt.title("Fig 7 - Milage at Failure", y=1.1)
plt.ylabel('Number of Failures', fontsize=12)
plt.xlabel('Miles at Failure (thousands)', fontsize=12)
plt.gcf()



In [ ]:

clean_df = cmpl_df[(cmpl_df["MILES"] >= 0) & (cmpl_df["MILES"] < 300000)]

plt.figure()
clean_df.plot(kind='scatter', x='MILES', y='OCCURENCES');
plt.title("Fig 5 - Number of Failures per Mile at Failure")
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Miles at Failure', fontsize=12)
plt.gcf()

#create_box_graph(clean_df, col_x='MILES', col_y='OCCURENCES', num_bins=10);



In [ ]:
import math;
matplotlib.style.use('ggplot')

curr_df = cmpl_df[["MILES", "OCCURENCES"]].dropna();
#curr_df = [(curr_df["MILES"] >= 980000) & (curr_df["MILES"] < 1000000)]

numBins = 20 + 1;
bins = py.linspace(0, 200000, numBins);
#bins = py.linspace(0, 50000, numBins);
#bin_df = pd.cut(curr_df['MILES'], bins, labels=bins[1:])

#print(bin_df)

binLabels = bins[1:];
binLabels = list(map(lambda x: math.floor(x / 1000), binLabels));
    
    
plt.figure()
curr_df.groupby(pd.cut(curr_df["MILES"], bins, labels=binLabels))["OCCURENCES"].sum().plot(kind='bar', color='#FF2200')
#bin_df.value_counts(sort=False).plot(kind='bar')
plt.title("Fig 8 - Number of Failure Occurrences by Mile", y=1.1);
plt.xlabel("Miles at Failure (thousands)", fontsize=12);
plt.ylabel("Number of Occurrences", fontsize=12);
plt.gcf()

#bins = py.linspace(0, 300000, numBins)
#bins = math.floor(bins)
#bins
#bin_df.value_counts(sort=False)

#cmpl_df['MILES'].value_counts()
#bin_df.describe()
#cmpl_df["MILES"].describe()

#bins = [900000, 1000000];
#cmpl_df.groupby(pd.cut(cmpl_df['MILES'], bins))["OCCURENCES"].describe()

#curr_df["OCCURENCES"].describe()
#curr_df[(curr_df["OCCURRENCES"] >= 980000) & (curr_df["MILES"] < 1000000)]

In [ ]:
cmpl_df = cmpl_df_new[cmpl_df_new["CMPLID"] > 1347687];

matplotlib.style.use('ggplot')

orange = '#FF6600';
blue = '#6666FF';

plotParams = {'labels':['No','Yes'], 'colors':[blue, orange], 'autopct':'%1.1f%%'}
titleOffset = 1.1;

plt.figure()
#cmpl_df["CRASH"].value_counts().plot.pie(labels=['No','Yes'], colors=[blue, orange], autopct='%1.1f%%');
cmpl_df["CRASH"].value_counts().plot.pie(**plotParams);
plt.title("Defects involving a Crash", y=titleOffset);
plt.axis('equal')
plt.ylabel("");
plt.show();

plt.figure()
((cmpl_df["INJURED"] > 0) | (cmpl_df["DEATHS"] > 0)).value_counts().plot.pie(**plotParams);
plt.title("Defects Involving Injuries or Death", y=titleOffset);
plt.axis('equal')
plt.ylabel("");
plt.show();

crashes_df = cmpl_df[cmpl_df["CRASH"] == 'Y'];

plt.figure()
#((cmpl_df["INJURED"] > 0) | (cmpl_df["DEATHS"] > 0)).value_counts().plot.pie();
((crashes_df["INJURED"] > 0) | (crashes_df["DEATHS"] > 0)).value_counts().plot.pie(**plotParams);
plt.title("Crashes Involving Injuries or Death", y=titleOffset);
plt.axis('equal')
plt.ylabel("");
plt.show();



In [ ]:
cmpl_df = load_data_file('CMPL', encoding='iso-8859-1')
#cmpl_df_new = load_data_file('CMPL_NEW', encoding='iso-8859-1')

print(cmpl_df["CMPLID"].max())
print(cmpl_df_new["CMPLID"].max())
print(cmpl_df_new["CMPLID"].max() - cmpl_df["CMPLID"].max())

cmpl_df_new[cmpl_df_new["CMPLID"] > 1347687]["CMPLID"].describe()

#cmpl_df['OCCURENCES'].describe()

#clean_col = cmpl_df[(cmpl_df["OCCURENCES"] >= -1000) & (cmpl_df["OCCURENCES"] < 20)]["OCCURENCES"]

#plt.figure()
#clean_col.hist(bins=20, color='#FF5500')
#clean_df.plot(kind='scatter', x='MILES', y='OCCURENCES');
#plt.title("Fig 7 - Milage at Failure", y=1.1)
#plt.ylabel('Number of Failures', fontsize=12)
#plt.xlabel('Miles at Failure (thousands)', fontsize=12)
#plt.gcf()

# Data cleaning for prediction

### Load the Complaints and Recalls datasets

In [4]:
# Load the complaints and recalls dataset.
cmpl_df = load_data_file('CMPL', encoding='iso-8859-1');
rcl_df = load_data_file('RCL', encoding='latin1');

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (15,22,26,32,33,34,35,36,38,39,40,41,42,43,44,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Filter and clean the Complaints dataset

In [12]:

# Create list of the columns we will use.
key_cols = ['MAKETXT', 'MODELTXT', 'YEARTXT']
numer_cols = ['INJURED', 'DEATHS', 'MILES', 'OCCURENCES', 'NUM_CYLS', 'VEH_SPEED']
bool_cols = ['CRASH', 'FIRE', 'POLICE_RPT_YN', 'ORIG_OWNER_YN', 'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'MEDICAL_ATTN', 'VEHICLES_TOWED_YN']
cat_cols = bool_cols + ['DRIVE_TRAIN', 'FUEL_SYS', 'FUEL_TYPE', 'TRANS_TYPE']
all_cols = key_cols + numer_cols + cat_cols

# Filter out rows with invalid make, model, or year.
# Also filter out any non-vehicles.
clean_df = cmpl_df[(cmpl_df['MAKETXT'] != 'UNKNOWN') &
                   (cmpl_df['MODELTXT'] != 'UNKNOWN') &
                   (cmpl_df['YEARTXT'] > 1900) &
                   (cmpl_df['YEARTXT'] < 2020) &
                   (cmpl_df['PROD_TYPE'] == 'V')][all_cols].copy()

# Convert certain columns to a numerical data type.
clean_df['NUM_CYLS'] = pd.to_numeric(clean_df['NUM_CYLS'], errors='coerce')

# Replace particular "bogus" values with NaN, which get filled later.
clean_df['INJURED'] = clean_df['INJURED'].replace(99, np.nan).fillna(0)
clean_df['DEATHS'] = clean_df['DEATHS'].replace(99, np.nan).fillna(0)

# Clamp the values for specific numerical columns.
clean_df['NUM_CYLS'] = clean_df['NUM_CYLS'].clip(4, 8)
clean_df['MILES'] = clean_df['MILES'].clip(0, 300000)
clean_df['OCCURENCES'] = clean_df['OCCURENCES'].clip(1, 10)
clean_df['VEH_SPEED'] = clean_df['VEH_SPEED'].clip(0, 100)

# Replace 'Y' and 'N' with 1 and 0 for boolean columns.
# Also convert them to numeric so we can average them.
# This will allow us to get a percentage of 'yes' for each vehicle type.
for column in bool_cols:
    clean_df[column] = clean_df[column].replace('Y', 1).replace('N', 0)
    clean_df[column] = pd.to_numeric(clean_df[column])

# Print out the resulting clean data set
clean_df[all_cols].head(10)

,MAKETXT,MODELTXT,YEARTXT,INJURED,DEATHS,MILES,OCCURENCES,NUM_CYLS,VEH_SPEED,CRASH,...,POLICE_RPT_YN,ORIG_OWNER_YN,ANTI_BRAKES_YN,CRUISE_CONT_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN,DRIVE_TRAIN,FUEL_SYS,FUEL_TYPE,TRANS_TYPE
0,MERCURY,COUGAR,1995.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GMC,SONOMA,1995.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NISSAN,MAXIMA,1987.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FORD,RANGER,1994.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GMC,SONOMA,1995.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MAZDA,626,1991.0,1.0,0.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LINCOLN,TOWN CAR,1994.0,0.0,0.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GMC,SONOMA,1995.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FORD,PROBE,1991.0,0.0,0.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,FORD,WINDSTAR,1995.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Group complaints by make/model/year and perform aggregate functions on the column values

Warning: this step takes a while to perform all the aggregate functions

In [116]:
# Group the dataset by vehicle type (make/model/year)
#grouped = clean_df.head(10000).groupby(key_cols)
grouped = clean_df.groupby(key_cols)

# Create a temp dataset containing the modes for each column.
clean_modes_df = clean_df.mode()

# Aggregate function whcih returns the mean of a series. If all values
# are null, then the specified default value is returned instead.
def aggMean(x, defaultValue):
    numNonNulls = x.dropna().shape[0]
    if numNonNulls == 0:
        return defaultValue
    return x.mean()

# Aggregate function which Returns the mode of a series. If all values are null,
# then the mode of this column over the entire dataset is returned instead.
def aggMode(x):
    numNonNulls = x.dropna().shape[0]
    if numNonNulls == 0:
        return clean_modes_df[x.name][0]
    return x.value_counts().index[0]

# Perform aggregate functions on the columns for each vehicle type.
grouped_df = grouped.agg(
            {'INJURED'           : lambda x: aggMean(x, 0),
             'DEATHS'            : lambda x: aggMean(x, 0),
             'MILES'             : lambda x: aggMean(x, 0),
             'OCCURENCES'        : lambda x: aggMean(x, 1),
             'NUM_CYLS'          : lambda x: aggMode(x),
             'VEH_SPEED'         : lambda x: aggMean(x, 0),
             'CRASH'             : lambda x: aggMean(x, 0),
             'FIRE'              : lambda x: aggMean(x, 0),
             'POLICE_RPT_YN'     : lambda x: aggMean(x, 0),
             'ORIG_OWNER_YN'     : lambda x: aggMean(x, 0),
             'ANTI_BRAKES_YN'    : lambda x: aggMean(x, 0),
             'CRUISE_CONT_YN'    : lambda x: aggMean(x, 0),
             'MEDICAL_ATTN'      : lambda x: aggMean(x, 0),
             'VEHICLES_TOWED_YN' : lambda x: aggMean(x, 0),
             'DRIVE_TRAIN'       : lambda x: aggMode(x),
             'FUEL_SYS'          : lambda x: aggMode(x),
             'FUEL_TYPE'         : lambda x: aggMode(x),
             'TRANS_TYPE'        : lambda x: aggMode(x)})

# Add in a column for the number of complaints for each vehicle type.
grouped_df['COMPLAINTS'] = grouped.size()

# Print out the grouped dataset.
grouped_df.head(10)


VEH_SPEED  VEHICLES_TOWED_YN  \
MAKETXT        MODELTXT          YEARTXT                                 
.              MALIBU            1998.0         0.0                0.0   
3 TON UTILITY  3EC14             2003.0        10.0                0.0   
4 TON UTILITY  4EC16W            2001.0        60.0                0.0   
                                 2002.0        55.0                0.0   
4-STAR         HORSE TRAILER     2006.0        20.0                0.0   
                                 2008.0         0.0                0.0   
                                 2012.0         0.0                0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0        35.0                0.0   
                                 2005.0         0.0                0.0   
               CARGO GOOSENECK   2005.0        50.0                0.0   

                                          MEDICAL_ATTN DRIVE_TRAIN  INJURED  \
MAKETXT        MODELTXT          YEARTXT                                      
.              MALIBU            1998.0            0.0         FWD      0.0   
3 TON UTILITY  3EC14             2003.0            0.0         FWD      0.0   
4 TON UTILITY  4EC16W            2001.0            0.0         FWD      0.0   
                                 2002.0            0.0         FWD      0.0   
4-STAR         HORSE TRAILER     2006.0            0.0         FWD      0.0   
                                 2008.0            0.0         FWD      0.0   
                                 2012.0            0.0         FWD      0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0            0.0         FWD      0.0   
                                 2005.0            0.0         FWD      0.0   
               CARGO GOOSENECK   2005.0            0.0         FWD      0.0   

                                          CRUISE_CONT_YN FUEL_SYS  OCCURENCES  \
MAKETXT        MODELTXT          YEARTXT                                        
.              MALIBU            1998.0              1.0       FI         1.0   
3 TON UTILITY  3EC14             2003.0              0.0       FI         1.0   
4 TON UTILITY  4EC16W            2001.0              0.0       FI         1.0   
                                 2002.0              0.0       FI         1.0   
4-STAR         HORSE TRAILER     2006.0              0.0       FI         1.0   
                                 2008.0              0.0       FI         1.0   
                                 2012.0              0.0       FI         5.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0              0.0       FI         1.0   
                                 2005.0              0.0       FI         1.0   
               CARGO GOOSENECK   2005.0              0.0       FI         1.0   

                                         FUEL_TYPE  ORIG_OWNER_YN  DEATHS  \
MAKETXT        MODELTXT          YEARTXT                                    
.              MALIBU            1998.0         GS            1.0     0.0   
3 TON UTILITY  3EC14             2003.0         GS            0.0     0.0   
4 TON UTILITY  4EC16W            2001.0         GS            0.0     0.0   
                                 2002.0         GS            0.0     0.0   
4-STAR         HORSE TRAILER     2006.0         GS            0.0     0.0   
                                 2008.0         GS            0.0     0.0   
                                 2012.0         GS            0.0     0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0         GS            0.0     0.0   
                                 2005.0         GS            0.0     0.0   
               CARGO GOOSENECK   2005.0         GS            0.0     0.0   

                                          POLICE_RPT_YN     MILES  FIRE  \
MAKETXT        MODELTXT          YEARTXT                                  
.              MALIBU            1998.0             0.0       0.0   0.0   
3 TON UTILITY  3EC14             2003.0             0.0   20000.0   0.0   
4 TON 

### Check which vehicle types exist in the Recalls dataset

In [117]:

# Create the set of vehicle types from the recalled dataset.
# Filter out rows with invalid make, model, or year.
# Also filter out any non-vehicles.
recalled_set = rcl_df[(rcl_df['MAKETXT'] != 'UNKNOWN') &
                      (rcl_df['MODELTXT'] != 'UNKNOWN') &
                      (rcl_df['YEARTXT'] > 1900) &
                      (rcl_df['YEARTXT'] < 2020) &
                      (rcl_df['RCLTYPECD'] == 'V')][key_cols].copy()

# Make the recalled set only have unique vehicle types,
# mostly for efficiency in the next step.
recalled_set = recalled_set[key_cols].groupby(key_cols).size()

# Now create the column in the complaints set which says
# whether a vehicle type is in the recalled set.
grouped_df['RECALLED'] = False
for index, row in grouped_df.iterrows():
    row['RECALLED'] = index in recalled_set.index


### TODO: more stuff, maybe with component types?

In [118]:
grouped_df.head(10)

#agg_df.loc['FORD', 'FOCUS']
#agg_df.loc['HONDA', 'CIVIC']


VEH_SPEED  VEHICLES_TOWED_YN  \
MAKETXT        MODELTXT          YEARTXT                                 
.              MALIBU            1998.0         0.0                0.0   
3 TON UTILITY  3EC14             2003.0        10.0                0.0   
4 TON UTILITY  4EC16W            2001.0        60.0                0.0   
                                 2002.0        55.0                0.0   
4-STAR         HORSE TRAILER     2006.0        20.0                0.0   
                                 2008.0         0.0                0.0   
                                 2012.0         0.0                0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0        35.0                0.0   
                                 2005.0         0.0                0.0   
               CARGO GOOSENECK   2005.0        50.0                0.0   

                                          MEDICAL_ATTN DRIVE_TRAIN  INJURED  \
MAKETXT        MODELTXT          YEARTXT                                      
.              MALIBU            1998.0            0.0         FWD      0.0   
3 TON UTILITY  3EC14             2003.0            0.0         FWD      0.0   
4 TON UTILITY  4EC16W            2001.0            0.0         FWD      0.0   
                                 2002.0            0.0         FWD      0.0   
4-STAR         HORSE TRAILER     2006.0            0.0         FWD      0.0   
                                 2008.0            0.0         FWD      0.0   
                                 2012.0            0.0         FWD      0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0            0.0         FWD      0.0   
                                 2005.0            0.0         FWD      0.0   
               CARGO GOOSENECK   2005.0            0.0         FWD      0.0   

                                          CRUISE_CONT_YN FUEL_SYS  OCCURENCES  \
MAKETXT        MODELTXT          YEARTXT                                        
.              MALIBU            1998.0              1.0       FI         1.0   
3 TON UTILITY  3EC14             2003.0              0.0       FI         1.0   
4 TON UTILITY  4EC16W            2001.0              0.0       FI         1.0   
                                 2002.0              0.0       FI         1.0   
4-STAR         HORSE TRAILER     2006.0              0.0       FI         1.0   
                                 2008.0              0.0       FI         1.0   
                                 2012.0              0.0       FI         5.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0              0.0       FI         1.0   
                                 2005.0              0.0       FI         1.0   
               CARGO GOOSENECK   2005.0              0.0       FI         1.0   

                                         FUEL_TYPE  ORIG_OWNER_YN  DEATHS  \
MAKETXT        MODELTXT          YEARTXT                                    
.              MALIBU            1998.0         GS            1.0     0.0   
3 TON UTILITY  3EC14             2003.0         GS            0.0     0.0   
4 TON UTILITY  4EC16W            2001.0         GS            0.0     0.0   
                                 2002.0         GS            0.0     0.0   
4-STAR         HORSE TRAILER     2006.0         GS            0.0     0.0   
                                 2008.0         GS            0.0     0.0   
                                 2012.0         GS            0.0     0.0   
4-STAR TRAILER CARGO BUMPER PULL 2000.0         GS            0.0     0.0   
                                 2005.0         GS            0.0     0.0   
               CARGO GOOSENECK   2005.0         GS            0.0     0.0   

                                          POLICE_RPT_YN     MILES  FIRE  \
MAKETXT        MODELTXT          YEARTXT                                  
.              MALIBU            1998.0             0.0       0.0   0.0   
3 TON UTILITY  3EC14             2003.0             0.0   20000.0   0.0   
4 TON 

# Prediction stuff

### Import required libraries

In [ ]:
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import ensemble
from sklearn.metrics import mean_squared_error